In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import scipy.misc

In [2]:
'''
출처
https://github.com/inyl/my_notebook/blob/master/open_cv/image_color_cluster.ipynb
https://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
'''

'\n출처\nhttps://github.com/inyl/my_notebook/blob/master/open_cv/image_color_cluster.ipynb\nhttps://www.pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/\n'

In [3]:
def plot_colors(hist, centroids):
    '''
    initialize the bar chart representing the relative frequency of each of the colors
    각 색의 빈도를 나타내는 바 차트를 초기화
    '''
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0

    # loop over the percentage of each cluster and the color of each cluster
    for (percent, color) in zip(hist, centroids):
        # plot the relative percentage of each cluster
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50),
                      color.astype("uint8").tolist(), -1)
        startX = endX

    # return the bar chart
    return bar

In [4]:
def centroid_histogram(clt):
    '''
    # grab the number of different clusters and create a histogram
    히스토그램 형식으로 색을 반환
    based on the number of pixels assigned to each cluster
    각 클러스터의 픽셀의 숫자를 기반으로 함
    '''
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()  # hist = hist/hist.sum()

    # return the histogram
    return hist

In [5]:
def image_color_cluster(image_path, k = 5):
    image = cv2.imread(image_path)
    # image의 shape을 찍어보면, height, width, channel 순으로 나옴
    # channel은 RGB를 말함
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # cv에서는 RGB가 아닌 BGR 순으로 나오기 때문에 순서를 RGB로 전환
    image = image.reshape((image.shape[0] * image.shape[1], 3))
    # shape의 0,1번째 즉, height와 width를 통합시킴
    
    clt = KMeans(n_clusters = k)  # 평균 알고리즘 KMeans
    clt.fit(image)

    hist = centroid_histogram(clt)
    bar = plot_colors(hist, clt.cluster_centers_)
    return bar

In [6]:
def dec_to_hex(color):
    if color < 16:
        return '0' + str(hex(int(color)).split('x')[1])
    else:
        return str(hex(int(color)).split('x')[1])

In [36]:
def read_real_color(filename, color_rank):
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    image_list = [str(list(image[i][k])) for i in range(len(image)) for k in range(len(image[0]))]
    image_unique = {}
    for d in image_list:
        if d not in image_unique:
            image_unique[d] = 1
        else:
            image_unique[d] += 1

    total_color_lists = sorted(image_unique.keys(), key=lambda x:image_unique[x], reverse=True)
    color_list = total_color_lists[color_rank]
    
    color_R = int(color_list.split('[')[1].split(']')[0].split(', ')[0])
    color_G = int(color_list.split('[')[1].split(']')[0].split(', ')[1])
    color_B = int(color_list.split('[')[1].split(']')[0].split(', ')[2])
    
    color_R = dec_to_hex(color_R)
    color_G = dec_to_hex(color_G)
    color_B = dec_to_hex(color_B)
    
    return str(color_R + color_G + color_B)

In [9]:
df_cafe = pd.read_csv('final_cafe_info_with_path.csv')
# df_cafe.head()

In [10]:
df_cafe = df_cafe.drop('Unnamed: 0', axis=1)
df_cafe.head()

,지역,지역구,카페명,주소,위도,경도,파일명
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697,ws_ep_01.png
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462,ws_ep_02.png
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774,ws_ep_03.png
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542,ws_ep_04.png
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534,ws_ep_05.png


In [44]:
for i in range(5):
    color_list = []
    for n in df_cafe.index:
        png = './cafe_color_result/' + df_cafe['파일명'][n]
        color_list.append(read_real_color(png, i))
    col_name = str(i+1) + '번 색'
    df_cafe[col_name] = color_list

In [48]:
df_cafe.head()

,지역,지역구,카페명,주소,위도,경도,파일명,1번 색,2번 색,3번 색,4번 색,5번 색
0,서서울,은평,스모어,서울 은평구 연서로29길 8-8,37.618952,126.919697,ws_ep_01.png,dbd6cc,c7b29a,231a14,ab8f73,644f3f
1,서서울,은평,필라멘트카페,서울 은평구 통일로 883,37.621753,126.919462,ws_ep_02.png,201d1e,9d7653,5f4939,cead7d,e7e4cf
2,서서울,은평,HUGA,서울 은평구 진관2로 19,37.634753,126.919774,ws_ep_03.png,36a17c,8ebda8,e5e7df,6f6c49,272b15
3,서서울,은평,YM Coffee Project,서울 은평구 연서로29길 21-8,37.619095,126.917542,ws_ep_04.png,cab9ab,e9dfd8,403126,af8b6d,84634c
4,서서울,은평,카페달력,서울 은평구 연서로18길 28-2,37.611930,126.920534,ws_ep_05.png,e8ddd1,d3c0af,b19c89,5a432c,917457


In [49]:
df_cafe.to_csv('cafe_color_tidy_data.csv')